# More information
Please see [our documentation](https://docs.google.com/document/d/16VR_w75hkn4Ss62phA6lGCqTWkkDpY9QASEU6NP0ARo/edit?usp=sharing) for more information about the data format.

In [5]:
from pathlib import Path
import os

import pandas as pd
import simuran
from simuran.loaders import nwb_loader
from pynwb.file import NWBFile

In [7]:
# Change these parameters to suit your needs
data_path = Path(r"I:\results")
working_dir = Path(r"I:")
nwb_file_csv_path = data_path / "processed_nwbfiles.csv"
os.chdir(working_dir)

In [8]:
recording_table = pd.read_csv(nwb_file_csv_path)
recordings = simuran.RecordingContainer.from_table(
    recording_table, nwb_loader.NWBLoader())
recordings.table.head()

,directory,filename,duration,rat,n_channels,sleep,maze,habituation_screening,treatment,light,...,unit_types,session,trial,passed,start1,choice1,end1,start2,choice2,end2
0,H:\SubRet_recordings_imaging\CSR6\screening_sm...,20032018_CSR6_screening_small sq_8.set,900.0,CSR6,32,0.0,small_sq,1.0,Control,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H:\SubRet_recordings_imaging\CSR6\screening_sm...,22022018_CSR6_screening_small sq_1.set,300.0,CSR6,32,0.0,small_sq,1.0,Control,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H:\SubRet_recordings_imaging\CSR6\screening_sm...,23022018_CSR6_screening_small sq_2.set,601.0,CSR6,32,0.0,small_sq,1.0,Control,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H:\SubRet_recordings_imaging\CSR6\screening_sm...,27022018_CSR6_screening_small sq_3.set,900.0,CSR6,32,0.0,small_sq,1.0,Control,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H:\SubRet_recordings_imaging\CSR6\small sq_bi...,22032018_CSR6_smallbig_small_1.set,1200.0,CSR6,32,0.0,small_sq,0.0,Control,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
example_recording = recordings.load(0)
nwb_data : NWBFile = example_recording.data
simuran.inspect(example_recording)

╭───────────────────────────────────── <class 'simuran.recording.Recording'> ─────────────────────────────────────╮
│ Describe a full recording session and holds data.                                                               │
│                                                                                                                 │
│ ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Recording(                                                                                                  │ │
│ │ │   attrs={                                                                                                 │ │
│ │ │   │   'directory': 'H:\\SubRet_recordings_imaging\\CSR6\\screening_small sq\\2003201'+1,                  │ │
│ │ │   │   'filename': '20032018_CSR6_screening_small sq_8.set',                                               │ │
│ │ │   │   'duration': 900.0,                                                                                  │ │
│ │ │   │   'rat': 'CSR6',                                                                                      │ │
│ │ │   │   'n_channels': 32,                                                                                   │ │
│ │ │   │   'sleep': 0.0,                                                                                       │ │
│ │ │   │   'maze': 'small_sq',                                                                                 │ │
│ │ │   │   'habituation_screening': 1.0,                                                                       │ │
│ │ │   │   'treatment': 'Control',                                                                             │ │
│ │ │   │   'light': nan,                                                                                       │ │
│ │ │   │   ... +18                                                                                             │ │
│ │ │   },                                                                                                      │ │
│ │ │   datetime=datetime.datetime(2023, 3, 10, 12, 34, 58, 907775),                                            │ │
│ │ │   tag=None,                                                                                               │ │
│ │ │   loader=NWBLoader(mode='r'),                                                                             │ │
│ │ │   source_file='results\\processed\\CSR6--screening_small sq--20032018--200320'+32,                        │ │
│ │ │   last_loaded_source='results\\processed\\CSR6--screening_small sq--20032018--200320'+32,                 │ │
│ │ │   data=root pynwb.file.NWBFile at 0x2322846157456                                                         │ │
│ │ Fields:                                                                                                     │ │
│ │   devices: {                                                                                                │ │
│ │ │   Bipolar electrodes 75um thick <class 'pynwb.device.Device'>,                                            │ │
│ │ │   Platinum-iridium wires 25um thick <class 'pynwb.device.Device'>                                         │ │
│ │   }                                                                                                         │ │
│ │   electrode_groups: {                                                                                       │ │
│ │ │   BE0 <class 'pynwb.ecephys.ElectrodeGroup'>,                                                             │ │
│ │ │   BE1 <class 'pynwb.ecephys.ElectrodeGroup'>,                                                             │ │
│ │ │   TT0 <class 'pynwb.ecephys.ElectrodeGroup'>,                                                             │ │
│ │ │   TT1 <class 'pynwb.ecephys.ElectrodeGroup'>,                                                             │ │
│ │ │   TT2 <class 'pynwb.ecephys.ElectrodeGroup'>,     

In [10]:
# Get the ID of this session
print(nwb_data.identifier)

ATNx_SUB_LFP--CSR6--screening_small sq--20032018--20032018_CSR6_screening_small sq_8


In [11]:
# Get the LFP
print(lfp := nwb_data.processing["ecephys"]["LFP"]["ElectricalSeries"])
print(lfp.data[:].T.shape)

ElectricalSeries pynwb.ecephys.ElectricalSeries at 0x2322847787952
Fields:
  comments: no comments
  conversion: 0.001
  data: <HDF5 dataset "data": shape (225000, 32), type "<f4">
  description: no description
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  filtering: Notch filter at 50Hz
  offset: 0.0
  rate: 250.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: volts

(32, 225000)


In [12]:
# Get the higher rate LFP
print(lfp := nwb_data.processing["high_rate_ecephys"]["LFP"]["ElectricalSeries"])
print(lfp.data[:].T.shape)

ElectricalSeries pynwb.ecephys.ElectricalSeries at 0x2322822471632
Fields:
  comments: no comments
  conversion: 0.001
  data: <HDF5 dataset "data": shape (1440000, 32), type "<f2">
  description: no description
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  filtering: Notch filter at 50Hz
  offset: 0.0
  rate: 1600.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: volts

(32, 1440000)


In [20]:
# Get the raw spiking data
nwb_data.processing["spikes"]["times"].to_dataframe().head()
nwb_data.processing["spikes"]["waveforms"].to_dataframe().head()

,tetrode_chan_id,num_spikes,timestamps
id,,,
0,1_ch1,1309,"[12.227354166666666, 13.055354166666667, 13.05..."
1,1_ch2,1309,"[12.227354166666666, 13.055354166666667, 13.05..."
2,1_ch3,1309,"[12.227354166666666, 13.055354166666667, 13.05..."
3,1_ch4,1309,"[12.227354166666666, 13.055354166666667, 13.05..."
4,2_ch1,252,"[3.1975208333333334, 6.7943125, 10.41441666666..."


,tetrode_chan_id,num_spikes,waveforms
id,,,
0,1_ch1,1309,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1_ch2,1309,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1_ch3,1309,"[43.359375, 48.046875, 43.359375, 32.8125, 28...."
3,1_ch4,1309,"[49.21875, 46.875, 39.84375, 33.984375, 31.640..."
4,2_ch1,252,"[3.90625, 0.9765625, 7.8125, 12.6953125, 10.74..."


In [30]:
# Get the position of the rat and running speed
print(nwb_data.processing["behavior"]["Position"]["SpatialSeries"])
print(nwb_data.processing["behavior"]["led_pixel_positions"])
print(nwb_data.processing["behavior"]["running_speed"])

SpatialSeries pynwb.behavior.SpatialSeries at 0x2322489356448
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (9000, 2), type "<f8">
  description: (x,y) position in camera
  interval: 1
  offset: 0.0
  reference_frame: (0,0) is top left corner
  resolution: -1.0
  timestamps: <HDF5 dataset "timestamps": shape (9000,), type "<f8">
  timestamps_unit: seconds
  unit: centimeters

led_pixel_positions pynwb.base.TimeSeries at 0x2322819743504
Fields:
  comments: no comments
  conversion: 0.3333333333333333
  data: <HDF5 dataset "data": shape (45000, 4), type "<u2">
  description: LED positions, note 1023 indicates untracked data. Order is Big LED x, Big LED y, Small LED x, Small LED y
  offset: 0.0
  rate: 50.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: centimeters

running_speed pynwb.base.TimeSeries at 0x2322819742544
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (9000,), type 

In [31]:
# normalised LFP
print(nwb_data.processing["normalised_lfp"]["LFP"]["ElectricalSeries"])
print(nwb_data.processing["normalised_lfp"]["LFP"]["ElectricalSeries"].data[:].T.shape)

ElectricalSeries pynwb.ecephys.ElectricalSeries at 0x2322846844432
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (225000, 32), type "<f8">
  description: no description
  electrodes: electrodes <class 'hdmf.common.table.DynamicTableRegion'>
  filtering: Notch filter at 50Hz
  offset: 0.0
  rate: 250.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: volts

(32, 225000)


In [37]:
regions = list(nwb_data.processing["average_lfp"].data_interfaces.keys())
for k in regions:
    nwb_data.processing["average_lfp"][k]

RSC_avg pynwb.base.TimeSeries at 0x2322819743408
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (225000,), type "<f8">
  description: A single averaged normalised LFP signal per brain region
  offset: 0.0
  rate: 250.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: V

SUB_avg pynwb.base.TimeSeries at 0x2322819741008
Fields:
  comments: no comments
  conversion: 1.0
  data: <HDF5 dataset "data": shape (225000,), type "<f8">
  description: A single averaged normalised LFP signal per brain region
  offset: 0.0
  rate: 250.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: V

In [40]:
nwb_data.processing["lfp_coherence"]["coherence_table"].to_dataframe().head()

,label,frequency,coherence
id,,,
0,SUB_RSC,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[0.24727093783546328, 0.21809291748874285, 0.1..."


In [45]:
nwb_data.processing["lfp_power"]["power_spectra"].to_dataframe().head()

,label,region,frequency,power,max_psd
id,,,,,
0,BE0_E0,SUB,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[-13.467952883966552, -6.324307171409834, -6.3...",3.084205e-13
1,BE0_E1,SUB,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[-7.812094387605991, -0.9720788605804719, -1.0...",1.655765e-13
2,BE1_E0,RSC,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[-13.46075222546008, -6.797650353425418, -6.96...",3.847032e-13
3,BE1_E1,RSC,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[-13.373212227458794, -6.717120717845411, -6.9...",3.839010e-13
4,TT0_E0,SUB,"[0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, ...","[-7.069660874901786, -0.32170457241395417, -1....",1.924688e-13


In [46]:
nwb_data.processing["speed_theta"]["speed_lfp_table"].to_dataframe().head()

,power,speed,region
id,,,
0,0.311566,12.0,RSC
1,0.429773,12.0,RSC
2,0.472681,10.0,RSC
3,0.472709,13.0,RSC
4,0.435072,23.0,RSC
